# Give me some credit: predict one input

In [83]:
import pandas as pd
import pickle

In [84]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier() # this class contains the implementation of the Random Forest learning technique for classification

In [85]:
from xgboost import XGBClassifier
model = XGBClassifier()

In [86]:
from sklearn.externals import joblib
from joblib import dump

In [87]:
val_prepared = pd.read_csv('csv/val_sk.csv', index_col=0)

In [88]:
val_prepared.head()
val_prepared = val_prepared.iloc[0:0]



In [89]:
val_prepared.head()

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs,Senior,IncomePerPerson,CombinedDefaulted,CombinedCreditLoans,MonthlyDebt,MonthlyBalance
Id,,,,,,,,,,,,,,,,,


In [90]:
data = {
    "RevolvingUtilizationOfUnsecuredLines": 0.01703559,
    "NumberOfDependents": 1,
    "DebtRatio": 0,
    "age": 42,
    "NumberOfOpenCreditLinesAndLoans": 6,
    "NumberRealEstateLoansOrLines": 1,
    "NumberOfTime30-59DaysPastDueNotWorse": 1,
    "NumberOfTime60-89DaysPastDueNotWorse": 0,
    "NumberOfTimes90DaysLate": 0
}


val_prepared = val_prepared.append([data])

In [91]:
val_prepared.head()

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs,Senior,IncomePerPerson,CombinedDefaulted,CombinedCreditLoans,MonthlyDebt,MonthlyBalance
0,0.017036,42.0,1,0.0,NaN,6,0,1,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Traitement des features

In [97]:
    # Age
    val_prepared.loc[val_prepared.age > 65, 'Senior'] = '1' 
    val_prepared.loc[val_prepared.age <= 65, 'Senior'] = '0'
    age_working = val_prepared.loc[(val_prepared["age"] >= 18) & (val_prepared["age"] < 65)]
    age_senior = val_prepared.loc[(val_prepared["age"] >= 65)]
    age_working_impute = age_working.MonthlyIncome.mean()
    age_senior_impute = age_senior.MonthlyIncome.mean()
    
    # MonthlyIncome
    val_prepared["MonthlyIncome"] = val_prepared["MonthlyIncome"].fillna(99999)
    val_prepared.loc[((df["age"] >= 18) & (val_prepared["age"] < 65)) & (val_prepared["MonthlyIncome"] == 99999), "MonthlyIncome"] = age_working_impute
    val_prepared.loc[(df["age"] >= 65) & (val_prepared["MonthlyIncome"] == 99999), "MonthlyIncome"] = age_senior_impute
    
    # Income Per Person
    val_prepared['IncomePerPerson'] = val_prepared['MonthlyIncome'] / ( val_prepared['NumberOfDependents'] + 1 )
    
    # Defaults
    val_prepared['CombinedDefaulted'] = (val_prepared['NumberOfTime30-59DaysPastDueNotWorse'] + \
                               val_prepared['NumberOfTime60-89DaysPastDueNotWorse'] + \
                               val_prepared['NumberOfTimes90DaysLate'])
        
    # Credit & Loans
    val_prepared["CombinedCreditLoans"] = val_prepared["NumberOfOpenCreditLinesAndLoans"] + \
                                val_prepared["NumberRealEstateLoansOrLines"]
    
    # Monthly Debt
    val_prepared['MonthlyDebt'] = val_prepared['DebtRatio'] * val_prepared['MonthlyIncome']
    
    # Monthly Balance
    val_prepared['MonthlyBalance'] = val_prepared['MonthlyIncome'] - val_prepared['MonthlyDebt']
    
    # Dependents
    val_prepared['NumberOfDependents'] = val_prepared['NumberOfDependents'].fillna(0)
    val_prepared["NumberOfDependents"] = val_prepared["NumberOfDependents"].astype('int64')

In [98]:
val_prepared.head()

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs,Senior,IncomePerPerson,CombinedDefaulted,CombinedCreditLoans,MonthlyDebt,MonthlyBalance
0,0.017036,42.0,1,0.0,NaN,6,0,1,0,1,NaN,0,NaN,1,7,NaN,NaN


Load model from disk

In [93]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

## Apply model to new set

In [94]:
target_column = 'SeriousDlqin2yrs'

X_val = val_prepared.drop(target_column, axis=1).values
y_val = val_prepared[target_column].values

y_val_proba = loaded_model.predict_proba(X_val)
y_val_proba = y_val_proba[:,1] # Avoir juste 1proba

print(y_val_proba)



[0.03836678]


In [95]:
print(f"Notre modèle estime la probabilité de défaut de remboursement du client à : {y_val_proba[0] * 100}% ")

Notre modèle estime la probabilité de défaut de remboursement du client à : 3.836677595973015% 
